In [1]:
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

In [2]:
# Loading the data, it comes from a lot of joined datasets
df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)
df.head()

# Drop information that cannot be substituted
def dropnas(df):
    df = df.loc[~df["FTHG"].isna() & ~df["FTAG"].isna()].copy()
    df = df.loc[~df["HTHG"].isna() & ~df["HTAG"].isna()].copy()
    df = df.loc[~df["HomeTeam"].isna() & ~df["AwayTeam"].isna()].copy()
    df = df.loc[~df["Div"].isna() & ~df["Date"].isna()].copy()
    return df

print("Nrows before dropping:", df.shape[0])
df = dropnas(df)
print("Nrows after dropping:", df.shape[0])

# Realistically, we can expect there to be less than 5 goals from each side in a match, but well cap it off at 10 for good measure
print("Nrows before dropping:", df.shape[0])
df = df.loc[~df["FTAG"].isna() & ~df["HTAG"].isna() & ~df["FTHG"].isna() & ~df["HTHG"].isna()].copy()
df = df.loc[(df["FTAG"] >= 0) & (df["FTAG"]<= 5) & (df["FTAG"] >= 0) & (df["FTHG"] <= 5)].copy()
print("Nrows after dropping:", df.shape[0])

# Let's see if we can fill in odds values
bet_columns = ["B365", "BS", "BW", "GB", "IW", "LB", "PS", "SO", "SB", "SJ", "SY", "VC", "WH"]
home_bet = [col+"H" for col in bet_columns]
draw_bet = [col+"D" for col in bet_columns]
away_bet = [col+"A" for col in bet_columns]

df["AvgH"] = df[home_bet].mean(axis=1)
df["AvgD"] = df[draw_bet].mean(axis=1)
df["AvgA"] = df[away_bet].mean(axis=1)

df["InvH"] = 1 / df["AvgH"]
df["InvD"] = 1 / df["AvgD"]
df["InvA"] = 1 / df["AvgA"]

df["OddsInv"] = df["InvH"] + df["InvD"] + df["InvA"]

# The good news is, that none of the odds sum are bellow 1, as that would mean a loss for the broker
# We can see that some brokers collect a higher premium for the bets, reaching almost 20% in some cases
print("Nrows before dropping:", df.shape[0])
df = df.loc[(df["OddsInv"]>1) & ~df["AvgH"].isna() & ~df["AvgD"].isna() & ~df["AvgA"].isna()]
print("Nrows after dropping:", df.shape[0])

# Now let's fill in the missing half-time and full-time results
def calculate_win(home_goals, away_goals):
    results = []
    for home, away in zip(home_goals, away_goals):
        if home < away:
            results.append("A")
        elif home > away:
            results.append("H")
        else:
            results.append("D")
    return results

df["HTR"] = calculate_win(df["HTHG"], df["HTAG"])
df["HTR"] = df["HTR"].map({"H": 1, "D": 0, "A": -1})
df["FTR"] = calculate_win(df["FTHG"], df["FTAG"])
df["FTR"] = df["FTR"].map({"H": 1, "D": 0, "A": -1})

print("Nrows before dropping:", df.shape[0])
df = df.loc[~df["HTR"].isna() & ~df["FTR"].isna()]
print("Nrows after dropping:", df.shape[0])

# Parse dates for later sampling
df["Date"] = df["Date"].apply(
        lambda date: datetime.strptime(date, "%d/%m/%y") if len(date)==8 else datetime.strptime(date, "%d/%m/%Y")
    )

df["Year"] = df["Date"].apply(lambda x: x.year)

df = df.loc[df["season"].isin([1819, 1920, 2021, 2122])].sort_values(by="Date", ascending=True).copy()
print("Final nrows:", df.shape[0])

C:\Users\vojte\AppData\Local\Temp\ipykernel_17256\1477887421.py:2: DtypeWarning: Columns (65,76,91,154,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)


Nrows before dropping: 155173
Nrows after dropping: 153086
Nrows before dropping: 153086
Nrows after dropping: 151390
Nrows before dropping: 151390
Nrows after dropping: 151037
Nrows before dropping: 151037
Nrows after dropping: 151037
Final nrows: 27069


In [3]:
chosen_columns = ["country", "Div", "Date", "season", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HY", "AY", "HR", "AR", "HS", "AS", "HST", "AST", "AvgH", "AvgD", "AvgA"]
df = df[chosen_columns].dropna().reset_index(drop=True).copy()
df["HST_ratio"] = df["HST"] / (df["HS"] + df["HST"])
df["AST_ratio"] = df["AST"] / (df["AS"] + df["AST"])
indexes = list(df.index)

In [4]:
def calculate_stats(country, country_stats, seasons=[1718, 1819, 1920, 2021, 2122]):
    master_df = pd.DataFrame()
    total_cumulative_stats = {}
    for season in seasons:
        cumulative_stats = {}
        home_values_for_tdf = {}
        away_values_for_tdf = {}
        tdf = df.loc[(df["country"] == country) & (df["season"]==season)].sort_values(by="Date", ascending=True).copy()
        for i in list(tdf.index):

            # Model home performance
            home = tdf.at[i, "HomeTeam"]
            ftr = tdf.at[i, "FTR"]
            if ftr==1:
                homepoints = 3
                awaypoints = 0
            elif ftr==-1:
                homepoints = 0
                awaypoints = 3
            else:
                homepoints = 1
                awaypoints = 1
            s1 = "H"
            cumulative_stats[home] = cumulative_stats.get(home, {})
            for col in ["FT0G", "0Y", "0R", "0S", "0ST"]:
                colname = col.replace("0", s1)
                cumulative_stats[home][colname] = cumulative_stats[home].get(colname, 0)
                home_values_for_tdf[colname+"_bm"] = home_values_for_tdf.get(colname+"_bm", []) + [cumulative_stats[home].get(colname, 0)]
                value = tdf.at[i, colname]
                cumulative_stats[home][colname] = cumulative_stats[home].get(colname, 0) + value
            
            cumulative_stats[home]["HST_ratio"] = cumulative_stats[home].get("HST_ratio", [0])
            home_values_for_tdf["HST_ratio"+"_bm"] = home_values_for_tdf.get("HST_ratio"+"_bm", []) + [np.array([cumulative_stats[home].get("HST_ratio", [0])]).mean()]
            hst_value = tdf.loc[i, "HST_ratio"]
            cumulative_stats[home]["HST_ratio"] = cumulative_stats[home].get("HST_ratio", [0]) + [hst_value]

            cumulative_stats[home]["HP"] = cumulative_stats[home].get("HP", 0)
            home_values_for_tdf["HP"+"_bm"] = home_values_for_tdf.get("HP"+"_bm", []) + [cumulative_stats[home].get("HP", 0)]
            cumulative_stats[home]["HP"] = cumulative_stats[home].get("HP", 0) + homepoints
            home_values_for_tdf["HP"+"_am"] = home_values_for_tdf.get("HP"+"_am", []) + [cumulative_stats[home].get("HP", 0)]
            
            # Model for away performance
            away = tdf.at[i, "AwayTeam"]
            s1 = "A"
            cumulative_stats[away] = cumulative_stats.get(away, {})
            for col in ["FT0G", "0Y", "0R", "0S", "0ST"]:
                colname = col.replace("0", s1)
                cumulative_stats[away][colname] = cumulative_stats[away].get(colname, 0)
                away_values_for_tdf[colname+"_bm"] = away_values_for_tdf.get(colname+"_bm", []) + [cumulative_stats[away].get(colname, 0)]
                value = tdf.at[i, colname]
                cumulative_stats[away][colname] = cumulative_stats[away].get(colname, 0) + value
            
            cumulative_stats[away]["AST_ratio"] = cumulative_stats[away].get("AST_ratio", [0])
            away_values_for_tdf["AST_ratio"+"_bm"] = away_values_for_tdf.get("AST_ratio"+"_bm", []) + [np.array([cumulative_stats[away].get("AST_ratio", [0])]).mean()]
            ast_value = tdf.loc[i, "AST_ratio"]
            cumulative_stats[away]["AST_ratio"] = cumulative_stats[away].get("AST_ratio", [0]) + [ast_value]

            cumulative_stats[away]["AP"] = cumulative_stats[away].get("AP", 0)
            away_values_for_tdf["AP"+"_bm"] = away_values_for_tdf.get("AP"+"_bm", []) + [cumulative_stats[away].get("AP", 0)]
            cumulative_stats[away]["AP"] = cumulative_stats[away].get("AP", 0) + awaypoints
            away_values_for_tdf["AP"+"_am"] = away_values_for_tdf.get("AP"+"_am", []) + [cumulative_stats[away].get("AP", 0)]



        for col in home_values_for_tdf.keys():
            tdf[col] = home_values_for_tdf[col]

        for col in away_values_for_tdf.keys():
            tdf[col] = away_values_for_tdf[col]

        if master_df.shape[0]==0:
            master_df = tdf.copy()
        else:
            master_df = pd.concat([master_df, tdf])

        total_cumulative_stats[season] = cumulative_stats

    country_stats["master_df"] = master_df.copy()
    country_stats["total_cumulative_stats"] = total_cumulative_stats
    return country_stats

In [5]:
def get_league_df(country_stats, total_cumulative_stats):
    flattened_data = []
    for season, teams in total_cumulative_stats.items():
        for team, stats in teams.items():
            row = {'season': season, 'team': team}
            row.update(stats)
            flattened_data.append(row)

    league_df = pd.DataFrame(flattened_data).copy()
    league_df["HST_ratio"] = league_df["HST_ratio"].apply(lambda x: np.mean(np.array(x)))
    league_df["AST_ratio"] = league_df["AST_ratio"].apply(lambda x: np.mean(np.array(x)))

    for col in ["FT0G", "0Y", "0R", "0S", "0ST", "0P"]:
        league_df[col.replace("0", "T")] = league_df[col.replace("0", "H")] + league_df[col.replace("0", "A")]

    league_df["TST_ratio"] = (league_df["HST_ratio"] + league_df["AST_ratio"]) / 2

    league_df = league_df[["season", "team", "FTTG", "TY", "TR", "TS", "TST", "TP", "TST_ratio"]].copy()

    country_stats["league_df"] = league_df.copy()
    return country_stats

In [6]:
def get_train_df(country, country_stats, league_df, train_seasons=[1920, 2021]):
    season_map = {1819: 1718, 1920: 1819, 2021:1920, 2122: 2021, 2223: 2122}
    train_df = df.loc[(df["country"] == country) & (df["season"].apply(lambda x: x in train_seasons))].sort_values(by="Date", ascending=True).copy()
    train_df = train_df[["season", "Date", "HomeTeam", "AwayTeam", "FTR", "AvgH", "AvgD", "AvgA", "Div"]].copy()
    train_df = pd.get_dummies(train_df, columns=["Div"], dtype=bool).copy()

    for col in list(league_df.columns[2:]):
        mapper = league_df.groupby(["team", "season"])[col].mean().to_dict()
        homevals = []
        awayvals = []
        for i in list(train_df.index):
            homevals.append(mapper.get((train_df.at[i, "HomeTeam"], season_map[train_df.at[i, "season"]]), None))
            awayvals.append(mapper.get((train_df.at[i, "AwayTeam"], season_map[train_df.at[i, "season"]]), None))
        train_df[col+"_H"] = homevals
        train_df[col+"_A"] = awayvals

    # train_df = pd.merge(left=train_df, right=current_stats, on=["Date", "HomeTeam", "AwayTeam"], how="left").dropna().copy()
    train_df = train_df.dropna().copy()

    for col in ["FTTG", "TY", "TR", "TS", "TST", "TP", "TST_ratio"]:
        train_df[col+"_diff"] = train_df[col+"_H"] - train_df[col+"_A"]

    train_df["OddsDiff"] = train_df["AvgH"] - train_df["AvgA"]
    country_stats["train_df"] = train_df.copy()
    return country_stats

In [15]:
import random

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, make_scorer

from sklearn.model_selection import KFold, cross_val_score

def train_model(country_stats, train_df, model):
    # chosen_columns = ["AvgH", "AvgD", "AvgA", "FTR"] # "OddsDiff", 
    chosen_columns = list(train_df.columns[4:])
    X_train = train_df[chosen_columns].copy()
    y_train = X_train.pop("FTR")

    # clf = GradientBoostingClassifier(n_estimators=100, max_depth=1, random_state=42)
    clf = model
    clf.fit(X_train, y_train)
    # clf = GradientBoostingClassifier(learning_rate=0.05, n_estimators=180, max_depth=1, random_state=42)
    # clf = KNeighborsClassifier(n_neighbors=10)
    k = 5
    kf = KFold(n_splits=k, random_state=42, shuffle=True)

    f1_scorer = make_scorer(f1_score, average="macro")
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=kf, scoring=f1_scorer)
    country_stats["f1_score_cv"] = scores.mean()

    country_stats["model_feature_importance"] = pd.DataFrame({"feature":X_train.columns ,"importance": clf.feature_importances_}).sort_values(by="importance", ascending=False)
    country_stats["model"] = clf

    return country_stats

In [55]:
country_seasons = {'france': [1819, 1920, 2021, 2122, 2223],
 'belgium': [1819, 1920, 2021, 2122, 2223],
 'germany': [1819, 1920, 2021, 2122, 2223],
 'england': [2021, 2122, 2223],
 'scotland': [1819, 1920, 2021, 2122, 2223],
 'turkey': [1819, 1920, 2021, 2122, 2223],
 'portugal': [1819, 1920, 2021, 2122, 2223],
 'netherlands': [2021, 2122, 2223],
 'spain': [2021, 2122, 2223],
 'italy': [2021, 2122, 2223],
 'greece': [1819, 1920, 2021, 2122, 2223]}

In [56]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
all_countries = {}

for country in df["country"].unique(): # df["country"].unique()
    season_length = country_seasons[country]
    country_stats = calculate_stats(
        country=country, country_stats={}, seasons=season_length
        )
    country_stats = get_league_df(
        country_stats=country_stats, total_cumulative_stats=country_stats["total_cumulative_stats"]
    )
    country_stats = get_train_df(
        country=country, country_stats=country_stats, league_df=country_stats["league_df"], train_seasons=season_length[1:]
    )
    # country_stats = get_test_df(
    #     country=country, country_stats=country_stats, league_df=country_stats["league_df"], test_seasons=season_length[-1:]
    # )
    country_stats = train_model(
        country_stats=country_stats, 
        train_df=country_stats["train_df"], 
        model = RandomForestClassifier(n_estimators=500, min_samples_split=20, max_depth=12, random_state=42, n_jobs=4)
    )
    all_countries[country] = country_stats

In [58]:
stats = []

for country in all_countries.keys():
    stats.append({
        "country": country, 
        "f1_cv": all_countries[country]["f1_score_cv"], 
        "df_size": all_countries[country]["train_df"].shape[0],
        "df_params": all_countries[country]["train_df"].shape[1]
        })

In [59]:
pd.DataFrame(stats).sort_values(by="f1_cv", ascending=False)

,country,f1_cv,df_size,df_params
6,portugal,0.468224,617,31
10,greece,0.456372,497,31
9,italy,0.436445,526,32
7,netherlands,0.432481,178,31
0,france,0.410769,1758,32
1,belgium,0.407800,631,31
3,england,0.394474,1842,34
5,turkey,0.393703,724,31
8,spain,0.387505,634,32
4,scotland,0.383922,1730,34


In [60]:
pred_df = pd.read_csv("test_ready.csv", index_col=0)
pred_df["season"] = [2223]*pred_df.shape[0]

In [61]:
pred_df.head()

,country,Div,Date,HomeTeam,AwayTeam,AvgH,AvgD,AvgA,season
0,belgium,B1,22/07/2022,Standard,Gent,3.995000,3.520000,1.898333,2223
1,belgium,B1,23/07/2022,Charleroi,Eupen,1.673333,3.933333,4.631667,2223
2,belgium,B1,23/07/2022,Kortrijk,Oud-Heverlee Leuven,2.373333,3.441667,2.838333,2223
3,belgium,B1,23/07/2022,Waregem,Seraing,2.191667,3.538333,3.023333,2223
4,belgium,B1,23/07/2022,St Truiden,St. Gilloise,4.076667,3.411667,1.898333,2223


In [102]:
def get_test_df(country, country_stats, league_df, test_seasons=[2223]):
    season_map = {1819: 1718, 1920: 1819, 2021:1920, 2122: 2021, 2223: 2122}
    test_df = pred_df.loc[(pred_df["country"] == country) & (pred_df["season"].apply(lambda x: x in test_seasons))].sort_values(by="Date", ascending=True).copy()
    test_df = test_df[["season", "Date", "HomeTeam", "AwayTeam", "AvgH", "AvgD", "AvgA", "Div"]].copy()
    test_df = pd.get_dummies(test_df, columns=["Div"], dtype=bool).copy()

    for col in list(league_df.columns[2:]):
        mapper = league_df.groupby(["team", "season"])[col].mean().to_dict()
        homevals = []
        awayvals = []
        for i in list(test_df.index):
            homevals.append(mapper.get((test_df.at[i, "HomeTeam"], season_map[test_df.at[i, "season"]]), None))
            awayvals.append(mapper.get((test_df.at[i, "AwayTeam"], season_map[test_df.at[i, "season"]]), None))
        test_df[col+"_H"] = homevals
        test_df[col+"_A"] = awayvals
    
    for col in list(league_df.columns[2:]):
        test_df[col+"_H"] = test_df[col+"_H"].fillna(test_df[col+"_H"].mean())
        test_df[col+"_A"] = test_df[col+"_A"].fillna(test_df[col+"_A"].mean())

    for col in ["FTTG", "TY", "TR", "TS", "TST", "TP", "TST_ratio"]:
        test_df[col+"_diff"] = test_df[col+"_H"] - test_df[col+"_A"]

    test_df["OddsDiff"] = test_df["AvgH"] - test_df["AvgA"]
    country_stats["test_df"] = test_df.copy()
    return country_stats

In [112]:
countries_test = {}

for country in ["england"]: # pred_df["country"].unique()
    country_stats = all_countries[country].copy()
    country_stats = get_test_df(
        country=country, country_stats=country_stats, league_df=country_stats["league_df"], test_seasons=[2223]
        )
    countries_test[country] = country_stats
    predict_dataset = country_stats["test_df"][country_stats["test_df"].columns[4:]].copy()
    predictions = all_countries[country]["model"].predict(predict_dataset)

    

In [110]:
countries_test["england"]["test_df"][countries_test["england"]["test_df"].columns[4:]]

,AvgH,AvgD,AvgA,Div_E0,Div_E1,Div_E2,Div_E3,FTTG_H,FTTG_A,TY_H,TY_A,TR_H,TR_A,TS_H,TS_A,TST_H,TST_A,TP_H,TP_A,TST_ratio_H,TST_ratio_A,FTTG_diff,TY_diff,TR_diff,TS_diff,TST_diff,TP_diff,TST_ratio_diff,OddsDiff
2067,2.491667,3.308333,2.715000,False,False,False,True,44.000000,54.000000,57.000000,60.000000,3.000000,2.000000,491.000000,511.000000,152.000000,182.000000,54.000000,47.000000,0.225088,0.240548,-10.0,-3.0,1.0,-20.0,-30.0,7.0,-0.015459,-0.223333
988,1.570000,3.798333,6.175000,False,True,False,False,53.000000,70.000000,82.000000,71.000000,0.000000,3.000000,527.000000,675.000000,181.000000,210.000000,69.000000,90.000000,0.233826,0.218390,-17.0,11.0,-3.0,-148.0,-29.0,-21.0,0.015436,-4.605000
2062,1.993333,3.376667,3.671667,False,False,False,True,52.000000,58.000000,70.000000,80.000000,4.000000,5.000000,608.000000,521.000000,183.000000,179.000000,58.000000,68.000000,0.219106,0.236931,-6.0,-10.0,-1.0,87.0,4.0,-10.0,-0.017825,-1.678333
2066,3.505000,3.418333,2.035000,False,False,False,True,55.230653,55.230653,74.690452,74.690452,2.920603,2.920603,520.628643,520.628643,176.011558,176.011558,59.447739,59.447739,0.233826,0.233826,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.470000
1522,2.170000,3.238333,3.340000,False,False,True,False,47.000000,61.000000,76.000000,70.000000,5.000000,7.000000,503.000000,487.000000,172.000000,180.000000,50.000000,40.000000,0.231622,0.257952,-14.0,6.0,-2.0,16.0,-8.0,10.0,-0.026330,-1.170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,1.123333,8.578333,21.641667,True,False,False,False,81.000000,43.000000,39.000000,76.000000,1.000000,6.000000,642.000000,438.000000,220.000000,138.000000,84.000000,39.000000,0.235191,0.215439,38.0,-37.0,-5.0,204.0,82.0,45.0,0.019752,-20.518333
466,1.986667,3.581667,3.691667,True,False,False,False,72.000000,43.000000,63.000000,62.000000,2.000000,2.000000,635.000000,478.000000,224.000000,169.000000,75.000000,40.000000,0.243747,0.244012,29.0,1.0,0.0,157.0,55.0,35.0,-0.000266,-1.705000
465,2.875000,3.241667,2.516667,True,False,False,False,67.000000,50.000000,91.000000,67.000000,4.000000,1.000000,540.000000,412.000000,199.000000,147.000000,79.000000,48.000000,0.259750,0.240926,17.0,24.0,3.0,128.0,52.0,31.0,0.018823,0.358333
464,4.010000,3.568333,1.911667,True,False,False,False,35.000000,57.000000,56.000000,75.000000,2.000000,2.000000,393.000000,508.000000,132.000000,187.000000,48.000000,58.000000,0.245308,0.259993,-22.0,-19.0,0.0,-115.0,-55.0,-10.0,-0.014684,2.098333
